# Artificial Neural Networks

### Importing the libraries

In [ ]:
import numpy as np
# don't need matplotlib
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv("Churn_Modelling.csv")

# independent variables (excluding the first three columns: entry number, customer ID, customer name)
# customer name is truly irrelevant to the model?
X = dataset.iloc[:, 3:-1].values
# dependent variable (variable to be predicted; rightmost column)
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

### Encoding Categorical Data

#### Label encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
# encoding just the gender column

In [ ]:
print(X)

#### One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

### Splitting the data into the training set and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Feature Scaling

In [ ]:
# Feature scaling is absolutely compulsory for all data in deep learning, even binary variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

## Part 2: Building the ANN

### Initializing the ANN

In [ ]:
# Keras used to be its own library but was integrated into TensorFlow as of 2.0
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
# there is no perfect answer for the best number of neurons in the hidden layer. It requires experimentation.
ann.add(tf.keras.layers.Dense(units=6, activation="relu")) # relu = rectifier

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) # sigmoid returns a probability of being either 0 or 1
# The outcome is binary, so one neuron is enough. For classification tasks you would need more


## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# adam = a sochastic gradient descent optimizer
# binary cross entropy is always used for binary dependent variables
# you can enter many metrics, but we are ony using one

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size= 32, epochs=100) # 32 is the default batch size value

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Homework: Use the ANN model to predict if the customer with the following information will leave the bank:

Geography: France  
Credit score: 600  
Gender: Male  
Age: 40  
Tenure: 3 years  
Balance: 60000  
Number of products: 2  
Does this customer have a credit card? Yes  
Is this customer an active member: yes  
Estimated Salary: $50000  
So, should we say goodbye to the customer?

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) # converts output above 0.5 to true
# Must have the right number of elements!

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1 )), 1)) 

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)